In [ ]:
pip install bs4

In [ ]:
import requests
from requests_oauthlib import OAuth1
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import json

In [ ]:
def get_data(cik):
    URL = f"https://www.edgarcompany.sec.gov/servlet/CompanyDBSearch?page=detailed&cik=000{cik}&main_back=2"
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "lxml") 

    dom = etree.HTML(str(soup))

    address = dom.xpath('/html/body/form/table/tr[3]/td/table/tr[2]/td/table/tr[8]/td[3]/text()')

    return ', '.join(address)

In [ ]:
# get all sec registered public company tickers
req_json_url = 'https://www.sec.gov/files/company_tickers.json'
result = requests.get(req_json_url).json()

In [ ]:
result = requests.get(req_json_url).json()
df_result = pd.DataFrame.from_dict(result.values())

In [ ]:
# query sec (EDGAR) site by CIK number 
address_list = []
for k,v in result.items():
    try:
        r = get_data(v.get('cik_str',''))
        address_list.append(r)
    except (KeyError,IndexError) as e:
        print(e)
        print(v)


In [ ]:
#https://companiesmarketcap.com/real-estate/largest-real-estate-companies-by-market-cap/
df = pd.read_csv('../files/real_estate_companies_by_marketcap.csv')
df_merge = pd.merge(df_result,df,right_on='Symbol',left_on='ticker')
df_final = pd.merge(df_merge,df_all[['title','address']],on='title')


In [ ]:
def yahoo_finance_industry(ticker):
    URL = f"https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}"
    page = requests.get(URL,headers = {'User-Agent': 'Mozilla/5.0'})

    soup = BeautifulSoup(page.content, "lxml") 

    dom = etree.HTML(str(soup))
    xpath = '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div/section/div[1]/div/div/p[2]/span[4]/text()'

    return dom.xpath(xpath)

In [ ]:
from random import randint
from time import sleep

In [ ]:
i = 0
for t in df_result.ticker:
#     sleep(randint(3,10))
    if t not in res_dict:
        industry = yahoo_finance(t)
        try:
            res_dict[t] = industry
            i+=1
            if i%10 == 0:
                print(f'Working on {t}')
        except err as e:
            print(f'Failed for {t} with {e}')
            continue
    


In [ ]:
def from_list(x):
    try:
        x = x[0]
    except:
         x = None
    return x    

In [ ]:
df_industry = pd.DataFrame(data=res_dict.items(),columns=['ticker','industry'])
df_industry['Industry'] = df_industry['industry'].apply(lambda x: from_list(x))
df_industry.to_excel('all_public_company_industries.xlsx',index=False)